In [1]:
class Config:
    """feel free to play with these hyperparameters during training"""
    dataset = "resource"  # change this to the right data name
    data_path = "./%s" % dataset
    checkpoint_dir = "checkpoint"
    decay_rate = 0.95
    decay_step = 1000
    n_topics = 50
    learning_rate = 0.00002
    vocab_size = 619
    n_stops = 22 
    lda_vocab_size = vocab_size - n_stops
    n_hidden = 200
    n_layers = 2
    projector_embed_dim = 100
    generator_embed_dim = n_hidden
    dropout = 1.0
    max_grad_norm = 1.0 #for gradient clipping
    total_epoch = 5
    init_scale = 0.075
    threshold = 0.5 #probability cut-off for predicting label to be 1
    forward_only = False #indicates whether we are in testing or training mode

    log_dir = './logs'

In [2]:
import pickle
import numpy as np

def save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)
    print(" [*] save %s" % path)

def load_pkl(path):
  with open(path,'rb') as f:
    obj = pickle.load(f)
    print(" [*] load %s" % path)
    return obj

def save_npy(path, obj):
  np.save(path, obj)
  print(" [*] save %s" % path)

def load_npy(path):
  obj = np.load(path)
  print(" [*] load %s" % path)
  return obj

In [3]:
import os
import itertools
import numpy as np
import nltk
import math

max_visit_size = 300


class PatientReader(object):
    def __init__(self, config):
        self.data_path = data_path = config.data_path

        self.vocab_path = vocab_path = os.path.join(data_path, "vocab.pkl")

        # use train data to build vocabulary
        if os.path.exists(vocab_path):
            self._load()
        else:
            pass

        self.vocab_size = config.vocab_size
        self.n_train_patients = math.ceil((len(self.X_train_data) + 0.0))
        self.n_valid_patients = math.ceil((len(self.X_valid_data) + 0.0))
        self.n_test_patients = math.ceil((len(self.X_test_data) + 0.0))

        self.lda_vocab_size = config.lda_vocab_size
        self.n_stops = config.n_stops

        self.idx2word = {v: k for k, v in self.vocab.items()} #needed to go from index to concept 

        print("vocabulary size: {}".format(self.vocab_size))
        print("number of training documents: {}".format(self.n_train_patients))
        print("number of validation documents: {}".format(self.n_valid_patients))
        print("number of testing documents: {}".format(self.n_test_patients))

    def _load(self):
        self.vocab = load_pkl(self.vocab_path)

        self.X_train_data = load_pkl(self.data_path + '/' + 'X_train' + '.pkl')
        self.Y_train_data = load_pkl(self.data_path + '/' + 'Y_train' + '.pkl')

        self.X_valid_data = load_pkl(self.data_path + '/' + 'X_valid' + '.pkl')
        self.Y_valid_data = load_pkl(self.data_path + '/' + 'Y_valid' + '.pkl')

        self.X_test_data = load_pkl(self.data_path + '/' + 'X_test' + '.pkl')
        self.Y_test_data = load_pkl(self.data_path + '/' + 'Y_test' + '.pkl')

    def get_data_from_type(self, data_type):
        if data_type == "train":
            X_raw_data = self.X_train_data
            Y_raw_data = self.Y_train_data
        elif data_type == "valid":
            X_raw_data = self.X_valid_data
            Y_raw_data = self.Y_valid_data
        elif data_type == "test":
            X_raw_data = self.X_test_data
            Y_raw_data = self.Y_test_data
        else:
            raise Exception(" [!] Unkown data type %s: %s" % data_type)

        return X_raw_data, Y_raw_data

    def get_Xc(self, data):
        """data is a patient...a sequence of visits
            so a list of lists...the outer list is of size T_patient
            the inner lists contain the concepts within each visit
        """
        patient = [concept for visit in data for concept in visit]
        patient = [x-1 for x in patient] 
        counts = np.bincount(patient, minlength=self.vocab_size)
        stops_flag = np.array(list(np.ones([self.lda_vocab_size], dtype=np.int32)) +
                              list(np.zeros([self.n_stops], dtype=np.int32)))

        return counts * stops_flag

    def get_X(self, data):
        """
        data is a list of lists of different length
        return an array of shape CxT where 
        entry Mij = ci if ci in visit j
        """
        T_patient = len(data)
        res = np.zeros([self.vocab_size, T_patient])
        for i in range(self.vocab_size):
            for j in range(T_patient):
                if (i+1) in data[j]:
                    res[i, j] = (i+1)

        return res

    def iterator(self, data_type="train"):
        """
        goes over the data and
        returns X, Xc, Y, and seq_len in a round robin
        seq_len is a vector of size C where each 
        entry is T_patient
        """
        X_raw_data, Y_raw_data = self.get_data_from_type(data_type)

        x_infos = itertools.cycle(([self.get_X(X_doc[:max_visit_size]), self.get_Xc(X_doc[:max_visit_size])]
                                   for X_doc in X_raw_data if X_doc != []))
        y_infos = itertools.cycle(([Y_doc[:max_visit_size], np.array([len(Y_doc[:max_visit_size])]*self.vocab_size)]
                                   for Y_doc in Y_raw_data if Y_doc != []))

        return x_infos, y_infos

In [157]:
import lasagne
from theano.tensor.shared_randomstreams import RandomStreams
class GaussianSampleLayer(lasagne.layers.MergeLayer):
    def __init__(self, mu, logsigma, maxlen, **kwargs):
        self.rng = RandomStreams(lasagne.random.get_rng().randint(1,2147462579))
        self.maxlen = maxlen
        super(GaussianSampleLayer, self).__init__([mu, logsigma], **kwargs)

    def get_output_shape_for(self, input_shapes):
        shape=(self.input_shapes[0][0] or inputs[0].shape[0],
               self.maxlen,
               self.input_shapes[0][1] or inputs[0].shape[1]) 
        #print(shape)
        return shape

    def get_output_for(self, inputs, deterministic=False, **kwargs):
        mu, logsigma = inputs
        shape=(self.input_shapes[0][0] or inputs[0].shape[0],
               self.maxlen,
                self.input_shapes[0][1] or inputs[0].shape[1])
        return mu + T.exp(logsigma) * self.rng.normal(shape)

In [159]:
from __future__ import print_function

import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import os
import time
import numpy as np
#from lasagne.layers.timefusion import MaskingLayer
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score, precision_recall_curve
#from lasagne.layers.theta import ThetaLayer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import average_precision_score as pr_auc

# Number of units in the hidden (recurrent) layer

# Number of training sequences in each batch


# All gradients above this will be clipped
GRAD_CLIP = 100
# How often should we check the output?



def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


def iterate_minibatches_listinputs(inputs, batchsize, shuffle=False):
    assert inputs is not None
    if shuffle:
        indices = np.arange(len(inputs[0]))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs[0]) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield [input[excerpt] for input in inputs]



def CONTENT(data_sets,N_HIDDEN = 200,embedsize = 100,n_topics = 50,num_epochs = 6,N_BATCH = 1):
    # Optimization learning rate
    LEARNING_RATE = theano.shared(np.array(0.001, dtype=theano.config.floatX))
    eta_decay = np.array(0.5, dtype=theano.config.floatX)
    # Min/max sequence length
    MAX_LENGTH = 300
    X_raw_data, Y_raw_data = data_sets.get_data_from_type("train")
    trainingAdmiSeqs, trainingMask, trainingLabels, trainingLengths, ltr = prepare_data(X_raw_data, Y_raw_data, vocabsize= 619, maxlen = MAX_LENGTH)
    Num_Samples, MAX_LENGTH, N_VOCAB = trainingAdmiSeqs.shape

    X_valid_data, Y_valid_data = data_sets.get_data_from_type("valid")
    validAdmiSeqs, validMask, validLabels, validLengths, lval  = prepare_data(X_valid_data, Y_valid_data, vocabsize= 619, maxlen = MAX_LENGTH)

    X_test_data, Y_test_data = data_sets.get_data_from_type("test")
    test_admiSeqs, test_mask, test_labels, testLengths, ltes = prepare_data(X_test_data, Y_test_data, vocabsize= 619, maxlen = MAX_LENGTH)
    alllength = sum(trainingLengths) + sum(validLengths) + sum(testLengths)
    print(alllength)
    eventNum = sum(ltr)+sum(lval)+sum(ltes)
    print(eventNum)



    print("Building network ...")
    
    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, max sequence length, number of features)
    l_in = lasagne.layers.InputLayer(shape=(N_BATCH, MAX_LENGTH, N_VOCAB))
    #l_label = lasagne.layers.InputLayer(shape=(N_BATCH, MAX_LENGTH, 1))

    # The network also needs a way to provide a mask for each sequence.  We'll
    # use a separate input layer for that.  Since the mask only determines
    # which indices are part of the sequence for each batch entry, they are
    # supplied as matrices of dimensionality (N_BATCH, MAX_LENGTH)
    l_mask = lasagne.layers.InputLayer(shape=(N_BATCH, MAX_LENGTH))
    
    
    #l_embed = lasagne.layers.DenseLayer(l_in, num_units=embedsize, b=None, W = W_embed, num_leading_axes=2)
    l_embed = lasagne.layers.DenseLayer(l_in, num_units=embedsize, b=None, num_leading_axes=2)
    #l_embed.params[l_embed.W].remove("trainable")
    #l_drop = lasagne.layers.dropout(l_embed)
    l_forward = lasagne.layers.GRULayer(
        l_embed, N_HIDDEN, mask_input=l_mask, grad_clipping=GRAD_CLIP,
        only_return_final=False)

    #l_forward = MaskingLayer([l_forward0, l_mask])
 
    l_1 = lasagne.layers.DenseLayer(l_in, num_units=N_HIDDEN, nonlinearity=lasagne.nonlinearities.rectify, num_leading_axes=2)
    l_2 = lasagne.layers.DenseLayer(l_1, num_units=N_HIDDEN, nonlinearity=lasagne.nonlinearities.rectify, num_leading_axes=2)
    mu = lasagne.layers.DenseLayer(l_2, num_units=n_topics, nonlinearity=None, num_leading_axes=1)# batchsize * n_topic
    log_sigma = lasagne.layers.DenseLayer(l_2, num_units=n_topics, nonlinearity=None, num_leading_axes=1)# batchsize * n_topic
    #l_theta = ThetaLayer([mu,log_sigma],maxlen=MAX_LENGTH)#batchsize * maxlen * n_topic
    l_theta = GaussianSampleLayer(mu,log_sigma,maxlen=MAX_LENGTH)

    l_B = lasagne.layers.DenseLayer(l_in, b=None, num_units=n_topics, nonlinearity=None, num_leading_axes=2)
    l_context = lasagne.layers.ElemwiseMergeLayer([l_B, l_theta],T.mul)
    l_context = lasagne.layers.ExpressionLayer(l_context, lambda X: X.mean(-1), output_shape="auto")

    l_dense0 = lasagne.layers.DenseLayer(
        l_forward, num_units=1, nonlinearity=None,num_leading_axes=2)
    l_dense1 = lasagne.layers.reshape(l_dense0, ([0], [1]))#batchsize * maxlen
    l_dense = lasagne.layers.ElemwiseMergeLayer([l_dense1, l_context],T.add)
    l_out0 = lasagne.layers.NonlinearityLayer(l_dense, nonlinearity=lasagne.nonlinearities.sigmoid)
    l_out = lasagne.layers.ExpressionLayer(lasagne.layers.ElemwiseMergeLayer([l_out0, l_mask],T.mul), lambda X:X+0.000001)



    target_values = T.matrix('target_output')
    target_values_flat = T.flatten(target_values)

    # lasagne.layers.get_output produces a variable for the output of the net
    network_output = lasagne.layers.get_output(l_out)
    # The network output will have shape (n_batch, maxlen); let's flatten to get a
    # 1-dimensional vector of predicted values
    predicted_values = network_output.flatten()
    # Our cost will be mean-squared error
    cost = lasagne.objectives.binary_crossentropy(predicted_values, target_values_flat)
    #kl_term = l_theta.klterm
    layer_outputs = lasagne.layers.get_output([mu, log_sigma])
    z_mu =  layer_outputs[0]
    z_ls =  layer_outputs[1]
    kl_term = 0.5 * T.sum(1 + 2*z_ls - T.sqr(z_mu) - T.exp(2 * z_ls))
    cost = cost.sum()-kl_term
    
    test_output = lasagne.layers.get_output(l_out, deterministic=True)

    #cost = T.mean((predicted_values - target_values)**2)
    # Retrieve all parameters from the network
    all_params = lasagne.layers.get_all_params(l_out)

    # Compute SGD updates for training
    print("Computing updates ...")
    updates = lasagne.updates.adam(cost, all_params, LEARNING_RATE)
    # Theano functions for training and computing cost
    print("Compiling functions ...")
    train = theano.function([l_in.input_var, target_values, l_mask.input_var],
                            cost, updates=updates)
    compute_cost = theano.function(
        [l_in.input_var, target_values, l_mask.input_var],cost)
    prd = theano.function([l_in.input_var, l_mask.input_var], test_output)
    #rnn_out = T.concatenate(l_theta.theta, lasagne.layers.get_output(l_forward0)[:,-1,:].reshape((N_BATCH, N_HIDDEN)),axis=1)
    #output_theta = theano.function([l_in.input_var, l_mask.input_var], [l_theta.theta, lasagne.layers.get_output(l_forward0)[:,-1,:].reshape((N_BATCH, N_HIDDEN))], on_unused_input='ignore')



    print("Training ...")
    try:
        for epoch in range(num_epochs):
            train_err = 0
            train_batches = 0
            start_time = time.time()
            thetas_train = []
            for batch in iterate_minibatches_listinputs([trainingAdmiSeqs, trainingLabels, trainingMask], N_BATCH,
                                                        shuffle=True):
                inputs = batch
                train_err += train(inputs[0], inputs[1], inputs[2])
                train_batches += 1
                #theta_train, rnnvec_train = output_theta(inputs[0], inputs[2])
                #rnnout_train = np.concatenate([theta_train, rnnvec_train], axis=1)
                #thetas_train.append(rnnout_train.flatten())
                #thetas_train.append(rnnvec_train.flatten())
                #if (train_batches+1)% 1000 == 0:
                #    print(train_batches)


            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))
            print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
 
            # After training, we compute and print the test error:
            test_err = 0

            test_batches = 0
            new_testlabels = []
            pred_testlabels = []
            thetas = []
            for batch in iterate_minibatches_listinputs([test_admiSeqs, test_labels, test_mask, testLengths], 1, shuffle=False):
                inputs = batch
                err = compute_cost(inputs[0], inputs[1], inputs[2])
                test_err += err
                leng = inputs[3][0]
                new_testlabels.extend(inputs[1].flatten()[:leng])
                pred_testlabels.extend(prd(inputs[0], inputs[2]).flatten()[:leng])
                #theta, rnnvec = output_theta(inputs[0], inputs[2])
                #rnnout = np.concatenate([theta, rnnvec],axis=1)
                #thetas.append(rnnout.flatten())
                #thetas.append(rnnvec.flatten())
                test_batches += 1
            test_auc = roc_auc_score(new_testlabels, pred_testlabels)
            test_pr_auc = pr_auc(new_testlabels, pred_testlabels)
 

            test_pre_rec_f1 = precision_recall_fscore_support(np.array(new_testlabels), np.array(pred_testlabels)>0.5, average='binary')
            test_acc = accuracy_score(np.array(new_testlabels), np.array(pred_testlabels)>0.5)
            print("Final results:")
            print("  test loss:\t\t{:.6f}".format(test_err / test_batches))
            print("  test auc:\t\t{:.6f}".format(test_auc))
            print("  test pr_auc:\t\t{:.6f}".format(test_pr_auc))
            print("  test accuracy:\t\t{:.2f} %".format(
                test_acc * 100))
            print("  test Precision, Recall and F1:\t\t{:.4f} \t\t{:.4f}\t\t{:.4f}".format(test_pre_rec_f1[0], test_pre_rec_f1[1], test_pre_rec_f1[2]))

    except KeyboardInterrupt:
        pass


def prepare_data(seqs, labels, vocabsize, maxlen=None):
    """Create the matrices from the datasets.
    This pad each sequence to the same lenght: the lenght of the
    longuest sequence or maxlen.
    if maxlen is set, we will cut all sequence to this maximum
    lenght.
    This swap the axis!
    """
    # x: a list of sentences
    lengths = [len(s) for s in seqs]

    eventSeq = []

    for seq in seqs:
        t = []
        for visit in seq:
            t.extend(visit)
        eventSeq.append(t)
    eventLengths = [len(s) for s in eventSeq]


    if maxlen is not None:
        new_seqs = []
        new_lengths = []
        new_labels = []
        for l, s, la in zip(lengths, seqs, labels):
            if l < maxlen:
                new_seqs.append(s)
                new_lengths.append(l)
                new_labels.append(la)
            else:
                new_seqs.append(s[:maxlen])
                new_lengths.append(maxlen)
                new_labels.append(la[:maxlen])
        lengths = new_lengths
        seqs = new_seqs
        labels = new_labels

        if len(lengths) < 1:
            return None, None, None

    n_samples = len(seqs)
    maxlen = np.max(lengths)

    x = np.zeros((n_samples, maxlen, vocabsize)).astype('int64')
    x_mask = np.zeros((n_samples, maxlen)).astype(theano.config.floatX)
    y = np.ones((n_samples, maxlen)).astype(theano.config.floatX)
    for idx, s in enumerate(seqs):
        x_mask[idx, :lengths[idx]] = 1
        for j, sj in enumerate(s):
            for tsj in sj:
                x[idx, j, tsj-1] = 1
    for idx, t in enumerate(labels):
        y[idx,:lengths[idx]] = t
        # if lengths[idx] < maxlen:
        #     y[idx,lengths[idx]:] = t[-1]

    return x, x_mask, y, lengths, eventLengths





In [140]:
C = Config()
datasets = PatientReader(C)
CONTENT(datasets)

 [*] load ./resource/vocab.pkl
 [*] load ./resource/X_train.pkl
 [*] load ./resource/Y_train.pkl
 [*] load ./resource/X_valid.pkl
 [*] load ./resource/Y_valid.pkl
 [*] load ./resource/X_test.pkl
 [*] load ./resource/Y_test.pkl
vocabulary size: 619
number of training documents: 2000
number of validation documents: 500
number of testing documents: 500
239887
665245
Building network ...
Computing updates ...
Compiling functions ...
Training ...
Epoch 1 of 6 took 557.066s
  training loss:		3081.478531
Final results:
  test loss:		3070.321072
  test auc:		0.800792
  test pr_auc:		0.642164
  test accuracy:		83.48 %
  test Precision, Recall and F1:		0.7097 		0.4435		0.5458
Epoch 2 of 6 took 558.447s
  training loss:		3079.344318
Final results:
  test loss:		3069.769506
  test auc:		0.798518
  test pr_auc:		0.646326
  test accuracy:		83.80 %
  test Precision, Recall and F1:		0.7897 		0.3764		0.5099
Epoch 3 of 6 took 570.463s
  training loss:		3078.690695
Final results:
  test loss:		3069.72228

In [160]:
#theano.config.exception_verbosity='high'
CONTENT(datasets,N_HIDDEN = 150,embedsize = 50,n_topics = 50,num_epochs = 1)

239887
665245
Building network ...
Computing updates ...
Compiling functions ...
Training ...
Epoch 1 of 1 took 438.894s
  training loss:		3081.710281
Final results:
  test loss:		3070.177727
  test auc:		0.799458
  test pr_auc:		0.638642
  test accuracy:		83.56 %
  test Precision, Recall and F1:		0.7274 		0.4244		0.5361


In [161]:
CONTENT(datasets,N_HIDDEN = 300,embedsize = 200,n_topics = 50,num_epochs = 1)

239887
665245
Building network ...
Computing updates ...
Compiling functions ...
Training ...
Epoch 1 of 1 took 1065.442s
  training loss:		3081.371253
Final results:
  test loss:		3069.850572
  test auc:		0.800690
  test pr_auc:		0.641014
  test accuracy:		83.52 %
  test Precision, Recall and F1:		0.7361 		0.4108		0.5273


In [162]:
CONTENT(datasets,N_HIDDEN = 100,embedsize = 100,n_topics = 100,num_epochs = 1)

239887
665245
Building network ...
Computing updates ...
Compiling functions ...
Training ...
Epoch 1 of 1 took 487.743s
  training loss:		3081.395941
Final results:
  test loss:		3069.658085
  test auc:		0.800571
  test pr_auc:		0.644693
  test accuracy:		83.63 %
  test Precision, Recall and F1:		0.7817 		0.3729		0.5049
